# Bitta Parking
Il Bitta Parking è un parcheggio a 5 livelli con numero variabile di posti, riservato esclusivamente agli Alfisti

In [24]:
# per importare la "cosa" che permette di leggere la data e l'ora a python

from datetime import datetime

In [15]:
parking = {"level1": {"total_parking_spaces": 10,
                      "taken_parking_spaces": 5},
            "level2": {"total_parking_spaces": 20,
                      "taken_parking_spaces": 7},
            "level3": {"total_parking_spaces": 39,
                      "taken_parking_spaces": 38},
            "level4": {"total_parking_spaces": 50,
                      "taken_parking_spaces": 50},
            "level5": {"total_parking_spaces": 5,
                      "taken_parking_spaces": 0}
            }

In [31]:
car = {"AX555LM": {"entry": datetime.now(), 
                   "level": "level1"},
        "CJ687LI": {"entry": datetime.now(), 
                   "level": "level3"}
        }


## Requisiti:
- occupiamo prima i piani inferiori;
- l'ultimo piano è riservato alla famiglia Bitta;
- i clienti pagano in base alla permanenza una quota proporzionata(€50/h);
- sono accettati esclusivamente possessori di Alfa Romeo e Camper.

In [17]:
# check if the vehicle is either an Alfa Romeo or a Camper 

def check_vehicle():
    model = input("Is this vehicle an Alfa Romeo or a Camper?  [Y/N]").lower()
    if model == "y":
        return True
    elif model == "n":
        return False
    else:
        check_vehicle() #in questo else, se la risposta non è né sì né n c'è una funzione ricorsiva, quinsi si ripropone la domanda fino a che la risposta non è o y o n
        

In [18]:
# check if the driver is a member of the Bitta Family

def is_Bitta():
    member = input("Are you a Bitta? [Y/N]").lower()
    if member == "y":
        return True
    elif member == "n":
        return False 
    else:
        is_Bitta()

In [19]:
def is_standard_spot_avaialble():
    for level, level_info in parking.items():
        if level == "level5":
            break
        total_s_parking_spaces = level_info["total_parking_spaces"]
        taken_s_parking_spaces = level_info["taken_parking_spaces"]
        Sspot_available = (total_s_parking_spaces - taken_s_parking_spaces) > 0

        if Sspot_available:
            return Sspot_available, level
        
    return Sspot_available, level # l'ultimo check che va a  fare se non ci sono posti disponibili in nessun livello, esce dal ciclo e ritorna false perché il quinto non viene considerato


In [20]:
# check if there's any avaialble parking space for the Bittas

def is_Bitta_spot_available():
    total_B_parking_spaces = parking["level5"]["total_parking_spaces"]
    taken_B_parking_spaces = parking["level5"]["taken_parking_spaces"]
    Bspot_available = (total_B_parking_spaces - taken_B_parking_spaces) > 0 
    return Bspot_available


In [33]:
def add_taken_spot(level):
    parking[level]["taken_parking_spaces"] += 1
    

In [35]:
def add_car(level):
    plate = input("Insert your license plate: ").upper()
    car[plate] = {} #aggiungiamo la targa all'auto che sta entrando nel parcheggio
    car[plate]["entry"] = datetime.now() # aggiungiamo l'orario di accesso della data macchina
    car[plate]["level"] = level # aggiungiamo il livello in cui l'auto deve andare
    add_taken_spot(level) # va ad aggiungere un posto preso
    

In [36]:
def remove_taken_spot(level):
    parking[level]["taken_parking_spaces"] -= 1
    

In [38]:
def remove_car(plate):
    plate_dict = car[plate] # seleziona le info sulle targhe
    entry_time = plate_dict["entry"] # prende l'orario di entrate
    level = plate_dict["level"] # prende il livello dove l'auto si trova 
    remove_taken_spot(level)

# LE COSE CHE CI MANCANO SONO:
- calcolare il prezzo da pagare
- rimuovere l'auto dal dizionario 
- farlo diventare qualcosa di eseguibile